# REG Capacity vs. Demand

Plots the historic and forecasted future project demand vs. team capacity for REG. This plot now uses only Forecast data with the exception of the available capacity from University Partner institutes, which is loaded from the file `wimbledon/vis/reg_capacity.csv`.

In Forecast:
* People should be allocated to the UNAVAILABLE project at 1 FTE (8hrs/day) to indicate their start and end dates, as well as fractional allocations for anyone who is available for less than 1 FTE for a period (if someone's long-term default allocation to REG is less than 1 FTE that should be changed using the capacity parameter in their profile instead).

* People should have one role out of "REG Associate", "REG Permanent", "REG FTC" (fixed term contract) and "University Partner" in their profile.

* Any team member on Forecast that should not be billable/allocated to projects, e.g. Fast Streamers, Project Managers, should have their capacity set to 0 hours per week in their profile.

* Unconfirmed project demand (e.g. projects with funder) can be indicated by allocating them to either the "Unconfirmed 01" or "Unconfirmed 02" placeholders. Similarly, projects that have been deferred can be indicated using the "Deferred" placeholder.

In [1]:
from wimbledon.vis.Visualise import Visualise
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
start_date = pd.datetime(2017,4,1)
end_date = pd.datetime(2022,3,31)
today = pd.datetime.now()

resample_freq = 'MS'

### Get Forecast Data

In [3]:
vis = Visualise(init_harvest=False)

Enpoint: whoami | Time: 0.5400255659999988 
Enpoint: people/460554 | Time: 0.4530126130000003 

AUTHENTICATED USER:
Jack Roberts jroberts@turing.ac.uk

CLIENTS
Enpoint: clients | Time: 0.450342924000001 
PROJECTS
Enpoint: projects | Time: 0.6887338559999989 
ROLES
Enpoint: roles | Time: 0.5672436959999985 
PEOPLE
Enpoint: people | Time: 0.5965969300000005 
PLACEHOLDERS
Enpoint: placeholders | Time: 0.46646691700000176 
MILESTONES
Enpoint: milestones | Time: 0.4162389510000004 
ASSIGNMENTS
Enpoint: assignments | Time: 0.6829626209999979 
DONE! (4.9s)


### Plot Demand vs. Capacity

In [6]:
vis.plot_demand_vs_capacity(start_date=start_date, 
                            end_date=end_date, 
                            freq=resample_freq,
                            today=today)

# save figure
plt.tight_layout()
plt.savefig('../data/figs/demand_vs_capacity.png',dpi=300)

IndexError: index 0 is out of bounds for axis 0 with size 0

### Table Demand vs. Client

In [5]:
client_meanfte = vis.table_client_demand(start_date=start_date, end_date=end_date)

display(client_meanfte)

# save table
client_meanfte.to_csv('../data/figs/demand_per_client.csv')

,2017-04-01 00:00:00,2018-04-01 00:00:00,2019-04-01 00:00:00,2020-04-01 00:00:00,2021-04-01 00:00:00
Data science at scale,0.2,0.9,0.0,0.0,0.0
Data science for science and the humanities,0.0,1.3,3.3,3.1,2.6
Data-centric engineering,0.8,2.6,4.5,5.4,0.5
Defence and security,1.7,3.2,3.7,2.0,2.0
External,0.0,0.0,0.0,0.0,0.0
Finance and economics,0.1,0.3,0.3,0.0,0.0
Health,0.4,0.9,2.5,1.4,0.5
Public Policy,0.1,0.4,0.2,0.0,0.0
REG Development Work,0.3,0.1,0.5,1.5,1.5
REG Management,0.9,1.8,2.6,2.6,2.6


## TODO
### High priority
1. **Demand / Capacity mismatch before 2018**
1. **Can we be confident 40 FTE is expected long run average? Perhaps x-FTE per programme (based on current less TF demand) + x-FTE per partner uni (support + TF demand + proj-specific demand)**

### Medium priority
1. **Add Scottish Enterprise demand back in time**
1. **Extend Third-party and Edinburgh Safe Haven capacity if demand is solid (check with JH). Should this be TPS or Hut23 project? If Hut23 move REG demand from June to this project.**
1. **Add AI programme demand?**

### Low priority
1. Split client table by funder.